In [ ]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.0 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


In [ ]:
%%writefile app.py
import streamlit as st
from transformers import FlaxAutoModelForSeq2SeqLM
from transformers import AutoTokenizer

# FUNCTIONS & GLOBAL VARS
MODEL_NAME_OR_PATH = "flax-community/t5-recipe-generation"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME_OR_PATH, use_fast=True)
model = FlaxAutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME_OR_PATH)

prefix = "items: "
# generation_kwargs = {
#     "max_length": 512,
#     "min_length": 64,
#     "no_repeat_ngram_size": 3,
#     "early_stopping": True,
#     "num_beams": 5,
#     "length_penalty": 1.5,
# }
generation_kwargs = {
    "max_length": 512,
    "min_length": 64,
    "no_repeat_ngram_size": 3,
    "do_sample": True,
    "top_k": 60,
    "top_p": 0.95
}

special_tokens = tokenizer.all_special_tokens
tokens_map = {
    "<sep>": "--",
    "<section>": "\n"
}
def skip_special_tokens(text, special_tokens):
    for token in special_tokens:
        text = text.replace(token, "")

    return text

def target_postprocessing(texts, special_tokens):
    if not isinstance(texts, list):
        texts = [texts]

    new_texts = []
    for text in texts:
        text = skip_special_tokens(text, special_tokens)

        for k, v in tokens_map.items():
            text = text.replace(k, v)

        new_texts.append(text)

    return new_texts

def generate_recipe(ingredients):
    texts = ""
    for key in ingredients:
        if ingredients[key]:
            texts = texts + key + ", "

    _inputs = texts if isinstance(texts, list) else [texts]
    inputs = [prefix + inp for inp in _inputs]
    inputs = tokenizer(
        inputs,
        max_length=256,
        padding="max_length",
        truncation=True,
        return_tensors="jax"
    )

    input_ids = inputs.input_ids
    attention_mask = inputs.attention_mask

    output_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        **generation_kwargs
    )
    generated = output_ids.sequences
    generated_recipe = target_postprocessing(
        tokenizer.batch_decode(generated, skip_special_tokens=False),
        special_tokens
    )
    return generated_recipe


# PAGE SETTINGS
st.set_page_config(page_title="fridge feast", page_icon=':black-box:', layout="wide")

# PAGE TITLE AND OPENING
with st.container():
    title_cols = st.columns([3.5,1])
    with title_cols[0]:
        st.title("fridge feast")
        st.text("welcome to fridge feast\n"
                "choose ingredients, click submit and start cooking")
        st.text("")
    with title_cols[1]:
        st.text("")
        st.image("https://img.hotimg.com/WhatsApp-Image-2024-02-26-at-17.46.29.jpeg", use_column_width=True)
        st.text("")
        st.text("")
        st.text("")
        st.text("")

# FORM
ingredients = {}
with st.container():
    with st.form("ingredients_form"):
        # CREATING INGREDIENTS DICT AND SPLITTING THE FORM TO INPUT AND OUTPUT
        tabs = st.tabs(["protein", "dairy", "carbs", "veggies"])

        with tabs[0]:
            # PROTEIN
            st.write("protein:")
            pro_cols = st.columns(2)
            ingredients["egg"] = pro_cols[0].checkbox("egg")
            ingredients["tofu"] = pro_cols[0].checkbox("tofu")
            ingredients["shrimp"] = pro_cols[0].checkbox("shrimp")
            ingredients["chicken"] = pro_cols[0].checkbox("chicken")
            ingredients["beef"] = pro_cols[0].checkbox("beef")
            ingredients["salmon"] = pro_cols[0].checkbox("fish")

            # Every form must have a submit button.
            submitted = st.form_submit_button("submit")

        with tabs[1]:
            # DAIRY
            st.write("dairy:")
            dairy_cols = st.columns(2)
            ingredients["butter"] = dairy_cols[0].checkbox("butter")
            ingredients["cream"] = dairy_cols[0].checkbox("cream")
            ingredients["cheese"] = dairy_cols[0].checkbox("cheese")
            ingredients["yogurt"] = dairy_cols[1].checkbox("yogurt")

        # RIGHT INPUT
        with tabs[2]:
            # CARBS
            st.text("carbs:")
            carb_cols = st.columns(2)
            ingredients["bread"] = carb_cols[0].checkbox("bread")
            ingredients["potato"] = carb_cols[0].checkbox("potato")
            ingredients["pasta"] = carb_cols[0].checkbox("pasta")
            ingredients["rice"] = carb_cols[0].checkbox("rice")

        with tabs[3]:
            # VEGGIES
            st.text("veggies:")
            veg_cols = st.columns([1, 1.2])
            ingredients["onion"] = veg_cols[0].checkbox("onion")
            ingredients["garlic"] = veg_cols[0].checkbox("garlic")
            ingredients["tomato"] = veg_cols[0].checkbox("tomato")
            ingredients["mushroom"] = veg_cols[1].checkbox("mushroom")
            ingredients["bell pepper"] = veg_cols[1].checkbox("bell pepper")
            ingredients["carrot"] = veg_cols[0].checkbox("carrot")



with st.container():
    st.text("")
    st.text("")
    st.text("")
    st.text("")
    st.text("")
    st.text("your recipe is here:")
    if submitted and any(ingredients.values()):
        # OUTPUT
        with st.spinner('waiting...'):
            generated = generate_recipe(ingredients)
            for text in generated:
                sections = text.split("\n")
                for section in sections:
                    section = section.strip()
                    if section.startswith("title:"):
                        section = section.replace("title:", "")
                        headline = "TITLE"
                    elif section.startswith("ingredients:"):
                        section = section.replace("ingredients:", "")
                        headline = "INGREDIENTS"
                    elif section.startswith("directions:"):
                        section = section.replace("directions:", "")
                        headline = "DIRECTIONS"

                    if headline == "TITLE":
                        st.text(f"{headline}: {section.strip().capitalize()}")
                    else:
                        section_info = [f"  {i+1}. {info.strip().capitalize()}" for i, info in enumerate(section.split("--"))]
                        st.text(f"{headline}:")
                        st.text("\n".join(section_info))

                print("-" * 130)
    else:
        # WAITING FOR USER SUBMISSION
        st.subheader("pick ingredients")

Writing app.py


In [ ]:
! wget -q -O - ipv4.icanhazip.com

# copy the code

35.237.12.127


In [ ]:
! streamlit run app.py & npx localtunnel --port 8501

# click on the 3rd link an paste the code to access the website




  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://35.237.12.127:8501

npx: installed 22 in 5.067s
your url is: https://tidy-carpets-sneeze.loca.lt
tokenizer_config.json: 100% 1.92k/1.92k [00:00<00:00, 8.84MB/s]
tokenizer.json: 100% 1.39M/1.39M [00:00<00:00, 11.7MB/s]
special_tokens_map.json: 100% 1.79k/1.79k [00:00<00:00, 8.73MB/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
config.json: 100% 1.56k/1.56k [00:00<00:00, 8.54MB/s]
flax_model.msgpack: 100% 892M/892M [00:10<00:00, 82.3MB/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
----------------------------------------------------------------------------------------------------------------------------------
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or traine